In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
import matplotlib.pyplot as plt
import seaborn as sns

spark = SparkSession.builder \
    .appName("Website Metrics") \
    .master('local') \
    .getOrCreate()

df = spark.read.parquet('/Users/mat/web_crawling_project/airflow/dags/data/gold_urls_categorized')
# Sample data for illustration
#data = [
#    ('site1.com', 'USA', 'News', False),
#    ('site2.com', 'UK', None, True),
#    ('site3.com', 'USA', 'News', False),
#    ('site4.com', 'Italy', 'Sports', True),
#    ('site5.com', 'Japan', 'News', False)
#]
#columns = ['url', 'country', 'category', 'is_ad']
#df = spark.createDataFrame(data, columns)

# 1. Distribution of Domains by Country

In [ ]:
# Calculate counts per country
country_distribution = df.groupBy("country").count().orderBy(F.desc("count"))
country_distribution.show()

# Plotting
country_distribution_pd = country_distribution.limit(10).toPandas()
plt.figure(figsize=(10, 6))
sns.barplot(x="country", y="count", data=country_distribution_pd)
plt.title("Distribution of Domains by Country")
plt.xlabel("Country")
plt.ylabel("Count of Domains")
plt.show()


# 2. Distribution of Categories

In [ ]:
# Calculate counts per category
category_distribution = df.groupBy("category").count().orderBy(F.desc("count"))
category_distribution.show()

# Plotting
category_distribution_pd = category_distribution.toPandas()
plt.figure(figsize=(12, 6))
sns.barplot(x="category", y="count", data=category_distribution_pd)
plt.title("Distribution of Categories")
plt.xlabel("Category")
plt.ylabel("Count of Domains")
plt.xticks(rotation=45)
plt.show()


# 3. Top Categories by Country

In [ ]:
# Calculate top categories by country
top_categories_by_country = df.groupBy("country", "category").count()
top_categories_by_country = top_categories_by_country.withColumn("rank", F.row_number().over(
    Window.partitionBy("country").orderBy(F.desc("count")))).filter(F.col("rank") <= 1)
top_categories_by_country.show()

# Plotting
top_categories_by_country_pd = top_categories_by_country.orderBy(F.desc("count")).limit(10).toPandas()
plt.figure(figsize=(12, 8))
sns.barplot(x="country", y="count", hue="category", data=top_categories_by_country_pd)
plt.title("Top Categories by Country")
plt.xlabel("Country")
plt.ylabel("Count of Domains")
plt.show()

# 4. Distribution of Ad-based Domains by Country

In [ ]:
# Filter for ad-based domains and calculate counts per country
ad_based_by_country = df.filter(F.col("is_ad") == True).groupBy("country").count().orderBy(F.desc("count"))
ad_based_by_country.show()

# Convert to Pandas for plotting
ad_based_by_country_pd = ad_based_by_country.limit(10).toPandas()

# Plotting
plt.figure(figsize=(8, 8))
plt.pie(ad_based_by_country_pd['count'], labels=ad_based_by_country_pd['country'], autopct='%1.1f%%')
plt.title("Distribution of Ad-based Domains by Country")
plt.show()


# 5. Percentage of Ad-Based Domains Over Total Domains for Each Country

In [ ]:
# Calculate total domains per country
total_domains_per_country = df.groupBy("country").count().withColumnRenamed("count", "total_count")

# Calculate ad-based domains per country
ad_domains_per_country = df.filter(F.col("is_ad") == True).groupBy("country").count().withColumnRenamed("count", "ad_count")

# Join the two DataFrames on 'country' to calculate the percentage
ad_percentage_per_country = total_domains_per_country.join(ad_domains_per_country, on="country", how="left")
ad_percentage_per_country = ad_percentage_per_country.withColumn(
    "ad_percentage", (F.col("ad_count") / F.col("total_count")) * 100
).fillna(0)  # Fill NA values with 0 where there are no ad-based domains

ad_percentage_per_country.show()

# Convert to Pandas for plotting
ad_percentage_per_country_pd = ad_percentage_per_country.select("country", "ad_percentage").orderBy(F.desc("ad_percentage")).limit(10).toPandas()

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x="country", y="ad_percentage", data=ad_percentage_per_country_pd)
plt.title("Percentage of Ad-Based Domains Over Total Domains for Each Country")
plt.xlabel("Country")
plt.ylabel("Percentage of Ad-Based Domains (%)")
plt.xticks(rotation=45)
plt.show()
